In [1]:
import numpy as np
import jax.numpy as jnp
import jax
import bayes3d as j
import bayes3d as b
import time
from PIL import Image
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
import cv2
import trimesh
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import genjax
from genjax import GenerativeFunction, ChoiceMap, Selection, trace

In [3]:
intrinsics = b.Intrinsics(
    height=300,
    width=300,
    fx=200.0, fy=200.0,
    cx=150.0, cy=150.0,
    near=0.001, far=6.0
)

b.setup_renderer(intrinsics)
b.RENDERER.add_mesh_from_file(os.path.join(j.utils.get_assets_dir(),"sample_objs/cube.obj"))

[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


Increasing frame buffer size to (width, height, depth) = (320, 320, 1024)


In [4]:
@genjax.gen
def g(x):
    m0 = genjax.trace("m0", genjax.bernoulli)(x)  # unsweetened
    return m0

In [14]:
g.assess(jax.random.PRNGKey(1),genjax.choice_map({"m0":1}),(0.1,))

(Array([0, 1], dtype=uint32), (1, Array(-2.3025851, dtype=float32)))

In [12]:
help(g.assess)

Help on method assess in module genjax._src.generative_functions.builtin.builtin_gen_fn:

assess(key: jaxtyping.UInt[Array, '...'], chm: genjax._src.core.datatypes.generative.ChoiceMap, args: tuple, **kwargs) -> tuple[jaxtyping.UInt[Array, '...'], tuple[typing.Any, typing.Union[float, jaxtyping.Float[Array, '...']]]] method of genjax._src.generative_functions.builtin.builtin_gen_fn.BuiltinGenerativeFunction instance



### Implementing HMM camera models

In [1]:
#how to make a compelling demo 